In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 3653 (delta 53), reused 67 (delta 27), pack-reused 3531
Receiving objects: 100% (3653/3653), 8.16 MiB | 9.60 MiB/s, done.
Resolving deltas: 100% (2278/2278), done.


In [0]:
! sudo pip install /content/fastText/.

Processing ./fastText
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2854934 sha256=e4c0dcafde5e2b4d82dac7a9ce76a20fec4fee0996c7aca388647f91392efc10
  Stored in directory: /tmp/pip-ephem-wheel-cache-k5gxg3oj/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


In [0]:
import fasttext

In [0]:
#breaks reviews up into individual words, tallies up word occurrences and extracts phrases where word appears.
#spacy and scattertext are not used because the results are decent without it and for companies with few reviews, compute time is instant.

import math
import json
import warnings
import pandas as pd
import numpy as np
from lxml import html
from requests import Session
from concurrent.futures import ThreadPoolExecutor as Executor
import requests
import re
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_colwidth', 1000)
base_url = "https://www.yelp.com/biz/" 
api_url = "/review_feed?sort_by=date_desc&start="
bid = 'OCTiJwvjoK81WoDwsTkFvA'
# bid = 'CBf1A4zCRNH-jsrJh48RsA'


class Scraper():
    def __init__(self):
        self.data = pd.DataFrame()

    def get_data(self, n, bid=bid):
        with Session() as s:
            with s.get(base_url+bid+api_url+str(n*20)) as resp: #makes an http get request to given url and returns response as json
                r = json.loads(resp.content) #converts json response into a dictionary
                _html = html.fromstring(r['review_list']) #loads from dictionary

                dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                df = pd.DataFrame([dates, reviews, ratings]).T

                self.data = pd.concat([self.data,df])

    def scrape(self): #makes it faster
        # multithreaded looping
        with Executor(max_workers=40) as e:
            list(e.map(self.get_data, range(10)))

s = Scraper()
s.scrape()
df = s.data
df = df.dropna()

df['word_segments_unpacked'] = df[1].apply(lambda x: x[1:-1].split(' '))#turn string comma separated list per word

df['word_segments_unpacked'] = df['word_segments_unpacked'].astype(str)
df['word_segments_unpacked'] = df['word_segments_unpacked'].apply(lambda x: ''.join([str(i) for i in x]))
phrase_count = df[['word_segments_unpacked', 2]]


s= phrase_count.apply(lambda x: pd.Series(x['word_segments_unpacked']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'word_segments_unpacked'

phrase_count = phrase_count.drop('word_segments_unpacked', axis=1).join(s)
phrase_count = pd.DataFrame(df['word_segments_unpacked'].str.split(',').tolist(), index=df[2]).stack()

phrase_count = phrase_count.reset_index()[[0, 2]] # var1 variable is currently labeled 0
phrase_count.columns = ['word_segments_unpacked', 'ratings'] # renaming var1
phrase_count = phrase_count.reset_index(drop=False)
replace_dict_phrase_count = {'[':'',']':'','-':'','!':'','.':'',' ':'',"'":''}
for key in replace_dict_phrase_count.keys():
  phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.replace(key, replace_dict_phrase_count[key])
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.lower()

stopwords = [')','(','\(','\xa0','0','1','2','3','4','5','6','7','8','9','/','$',"'d","'ll","'m",'+','maybe','from','first','here','only','put','where','got','sure','definitely','food','yet','our','go','since','really','very','two',"n't",'with','if',"'s",'which','came','all','me','(',')','makes','make','were','immediately','get','been','ahead','also','that','one','have','see','what','to','we','had','.',"'re",'it','or','he','she','we','us','how','went','no','"','of','has','by','bit','thing','place','so','ok','and','they','none','was','you',"'ve",'did','be','and','but','is','as','&','you','has','-',':','and','had','was','him','so','my','did','would','her','him','it','is','by','bit','thing','place','[',']','while','check-in','=','= =','want', 'good','husband', 'want','love','something','your','they','your','cuz','him',"i've",'her','told', 'check', 'i"m', "it's",'they', 'this','its','they','this',"don't",'the',',', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.']
def filter_stopwords(text):
  for i in str(text):
    if i not in stopwords:
      return str(text)

#if item in stopwords list partially matches, delete, single letters like 'i' would be deleted from inside individual words if in list
phrase_count = phrase_count[~phrase_count['word_segments_unpacked'].isin(stopwords)]
#if the following words fully matches, filter out
full_match_list = ['i','a','an','am','at','are','in','on','for','','\xa0\xa0','\xa0','\(']
phrase_count = phrase_count[~phrase_count['word_segments_unpacked'].isin(full_match_list)]

#pivot table ratings
phrase_count_pivot = pd.pivot_table(phrase_count, index='word_segments_unpacked', columns='ratings', aggfunc='count', fill_value=0)
phrase_count_pivot.columns = [''.join(col).strip() for col in phrase_count_pivot.columns.values]#flatten index levels part 1
phrase_count_pivot = pd.DataFrame(phrase_count_pivot.to_records())#flatten index levels part 2

#if there are no _# star reviews, add a column of zeros
required_column_names = ['index1.0 star rating', 'index2.0 star rating','index3.0 star rating','index4.0 star rating','index5.0 star rating']
for i in required_column_names:
  if i not in phrase_count_pivot.columns:
    phrase_count_pivot[i] = 0
phrase_count_pivot.sample(10)

#replace the original count by getting an exaggerated scaled tally of reviews to calculate score
phrase_count_pivot['index1.0 star rating'] = phrase_count_pivot['index1.0 star rating']*(-2)
phrase_count_pivot['index2.0 star rating'] = phrase_count_pivot['index2.0 star rating']*(-1)
phrase_count_pivot['index3.0 star rating'] = phrase_count_pivot['index3.0 star rating']*(-0.1)
phrase_count_pivot['index4.0 star rating'] = phrase_count_pivot['index4.0 star rating']*(1)
phrase_count_pivot['index5.0 star rating'] = phrase_count_pivot['index5.0 star rating']*(2)

#get a total score from the sum of exaggerated scores
phrase_count_pivot['score'] = phrase_count_pivot['index1.0 star rating'] + phrase_count_pivot['index2.0 star rating'] + phrase_count_pivot['index3.0 star rating'] + phrase_count_pivot['index4.0 star rating'] + phrase_count_pivot['index5.0 star rating']

phrase_count_pivot['score'] = phrase_count_pivot['score'].div(phrase_count_pivot['score'].max(), axis=0)#normalize
phrase_count_pivot['score'] = phrase_count_pivot['score'].round(decimals=4)#round to 4 decimal places
phrase_count_pivot = phrase_count_pivot.sort_values(by=('score'), ascending=False)



In [0]:
phrase_count_pivot['word_segments_unpacked'] = phrase_count_pivot['word_segments_unpacked'].str.replace('\(', '')
phrase_count_pivot['word_segments_unpacked'] = phrase_count_pivot['word_segments_unpacked'].str.replace('\\', '')
phrase_count_pivot['word_segments_unpacked'] = phrase_count_pivot['word_segments_unpacked'].str.replace('(', '')
phrase_count_pivot['word_segments_unpacked'] = phrase_count_pivot['word_segments_unpacked'].str.replace(')', '')#without these, errors incurr


worst_terms_list = [] 
top_terms_list = []
x,y = phrase_count_pivot.shape#tuple unpacking to get the length of the dataframe
for i in range(math.ceil(x/2)):
# for i in range(0,30):
  try:
    new_df = df[df[1].str.contains(phrase_count_pivot['word_segments_unpacked'].iloc[i])]#if word appears in review, create a dataframe with each row being the word occurring in a different review
    neg_first_df = new_df.sort_values(by=2, ascending=True)#rank the dataframe with worst reviews first
    pos_first_df = new_df.sort_values(by=2, ascending=False)#rank the dataframe with most positive reviews first
    if neg_first_df[1].iloc[0] not in worst_terms_list:#get the lowest star rating review
      worst_terms_list.append(neg_first_df[1].iloc[0])#prevent duplicates
    if pos_first_df[1].iloc[0] not in top_terms_list:#get the highest star rating review
      top_terms_list.append(pos_first_df[1].iloc[0])
  except IndexError as e:
    pass

In [0]:
negative_list = []
for i in range(-30,0):#take the worst 30 terms
  for list_of_words in worst_terms_list:
    word_list = list_of_words.split(' ')
    for word in word_list:
      word = word.replace(',','')
      word = word.replace('.','')
      try: 
        if phrase_count_pivot['word_segments_unpacked'].iloc[i] == word: #find word occurrence in original comma separated word list of reviews
          try:
            index = word_list.index(word)
            string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
            replace_dict_string_from_phrases= {'\xa0':'',' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
            for key in replace_dict_string_from_phrases.keys():
              string_from_phrases=string_from_phrases.replace(key, replace_dict_string_from_phrases[key])
            negative_list.append(string_from_phrases)
          except ValueError as e:
            pass
      except IndexError as e:#if there are less than 30 words after stopword filtering, just get the first word and its occurrence in the original review
        if phrase_count_pivot['word_segments_unpacked'].iloc[0] == word:
          try:
            index = word_list.index(word)
            string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
            replace_dict_string_from_phrases= {'\xa0':'',' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
            for key in replace_dict_string_from_phrases.keys():
              string_from_phrases=string_from_phrases.replace(key, replace_dict_string_from_phrases[key])
            negative_list.append(string_from_phrases)
          except ValueError as e:
            pass
negative_df = pd.DataFrame(negative_list)
negative_df = negative_df.reset_index(drop=False)
negative_df = negative_df.rename(columns={'index':'score', 0 : 'term'})
negative_df = negative_df.drop_duplicates(subset='term')
x,y = negative_df.shape#tuple unpacking to get the length of the dataframe
if x < 10:
  for i in range(-40,-30):
    for list_of_words in worst_terms_list:
      word_list = list_of_words.split(' ')
      for word in word_list:
        word = word.replace(',','')
        word = word.replace('.','')
        try:
          if phrase_count_pivot['word_segments_unpacked'].iloc[i] == word:
            try:
              index = word_list.index(word)
              string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
              replace_dict = {'\xa0':'',' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
              for key in replace_dict.keys():
                string_from_phrases=string_from_phrases.replace(key, replace_dict[key])
              negative_list.append(string_from_phrases)
            except ValueError as e:
              pass
        except IndexError as e:
          if phrase_count_pivot['word_segments_unpacked'].iloc[0] == word:
            try:
              index = word_list.index(word)
              string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
              replace_dict_string_from_phrases= {'\xa0':'',' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
              for key in replace_dict_string_from_phrases.keys():
                string_from_phrases=string_from_phrases.replace(key, replace_dict_string_from_phrases[key])
              negative_list.append(string_from_phrases)
            except ValueError as e:
              pass
negative_df_addon = pd.DataFrame(negative_list)
negative_df_addon = negative_df_addon.reset_index(drop=False)
negative_df_addon = negative_df_addon.rename(columns={'index':'score', 0 : 'term'})
negative_df = pd.concat([negative_df, negative_df_addon])
negative_df = negative_df.head(10)

In [0]:
positive_list = []
for i in range(0,30):
  for list_of_words in top_terms_list:
    word_list = list_of_words.split(' ')
    for word in word_list:
      word = word.replace(',','')
      word = word.replace('.','')
      try: 
        if phrase_count_pivot['word_segments_unpacked'].iloc[i] == word:
          try:
            index = word_list.index(word)
            string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
            replace_dict = {'\xa0':'',' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
            for key in replace_dict.keys():
              string_from_phrases=string_from_phrases.replace(key, replace_dict[key])
            negative_list.append(string_from_phrases)
          except ValueError as e:
            pass
      except IndexError as e:
        if phrase_count_pivot['word_segments_unpacked'].iloc[0] == word:
          try:
            index = word_list.index(word)
            string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
            replace_dict = {'\xa0':'',' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
            for key in replace_dict.keys():
              string_from_phrases=string_from_phrases.replace(key, replace_dict[key])
            negative_list.append(string_from_phrases)
          except ValueError as e:
            pass
  positive_list.append(string_from_phrases)
positive_df = pd.DataFrame(positive_list)
positive_df = positive_df.reset_index(drop=False)
positive_df = positive_df.rename(columns={'index':'score', 0 : 'term'})
positive_df = positive_df.drop_duplicates(subset='term')
x,y = positive_df.shape#tuple unpacking to get the length of the dataframe
for i in range(30,40):
  for list_of_words in top_terms_list:
    word_list = list_of_words.split(' ')
    for word in word_list:
      word = word.replace(',','')
      word = word.replace('.','')
      try:
        if phrase_count_pivot['word_segments_unpacked'].iloc[i] == word:
          try:
            index = word_list.index(word)
            string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
            replace_dict = {'\xa0':'',' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
            for key in replace_dict.keys():
              string_from_phrases=string_from_phrases.replace(key, replace_dict[key])
            negative_list.append(string_from_phrases)
          except ValueError as e:
            pass
      except IndexError as e:
        if phrase_count_pivot['word_segments_unpacked'].iloc[0] == word:
          try:
            index = word_list.index(word)
            string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
            replace_dict = {'\xa0':'',' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
            for key in replace_dict.keys():
              string_from_phrases=string_from_phrases.replace(key, replace_dict[key])
            negative_list.append(string_from_phrases)
          except ValueError as e:
            pass
positive_df_addon = pd.DataFrame(negative_list)
positive_df_addon = positive_df_addon.reset_index(drop=False)
positive_df_addon = positive_df_addon.rename(columns={'index':'score', 0 : 'term'})
positive_df = pd.concat([positive_df, positive_df_addon])
positive_df = positive_df.head(10)

In [0]:
phrase_count_pivot.head(2)

,word_segments_unpacked,index1.0 star rating,index2.0 star rating,index3.0 star rating,index4.0 star rating,index5.0 star rating,score
778,great,-4,-3,-0.6,15,80,1.0000
888,italian,-8,-1,-0.9,12,80,0.9394


In [0]:
df.to_csv(r'pandas.txt', header=None, index=None, sep=' ', mode='a')

In [0]:
model = fasttext.train_unsupervised('pandas.txt')

In [0]:
negative_df.head(2)

,score,term
0,0,restaurant is a landmark - home to the most horrible restaurant host in the known universe He seated my companion and I at a small
1,1,has got to be the worst Italian restaurant i have ever ate in in my life. three of us decided to just order appetizers but


In [0]:
snippet = positive_df.head(3)

In [0]:
similar_list = []

for i in positive_df['term']:
  model.get_nearest_neighbors(i)
  for i in (model.get_nearest_neighbors(i)):
    x,y = i
    similar_list.append(y)
print(similar_list)

['fantastic!', 'go', 'area.', "It's", 'expect', 'favorite', 'fast', 'far', 'reasonably', 'Prices', 'expect', 'reasonably', 'inviting.', 'area.', 'expectations', 'beginning', 'outstanding.', 'reasonable', 'fantastic!', 'Prices', 'Tiramisu', 'Gnocchi', 'portions.', 'portions', 'gnocchi', 'Very', '"Good', 'it!', 'authentic.', 'Absolutely', 'expect', 'reasonable', 'reasonably', 'eat', 'starts', 'cute', 'inviting', 'seating.', 'welcoming', 'beginning', 'authentic.', '"Good', 'portions', 'portions.', 'atmosphere.', 'Atmosphere', 'Very', 'portions,', 'food!', 'Beautiful', '\xa0Service', 'priced', 'service,', '"Service', 'atmosphere.', 'nice', 'service.', 'great,', 'great.', 'Service', 'gives', 'unbeatable', "Don't", 'heart', 'visit', 'point', 'lots', 'annual', 'guy', 'Overall,', 'miss', 'gem', 'Definitely', 'feast', 'return."', 'fantastic."', 'return', 'back."', 'again!"', 'customer', 'Tiramisu', 'Gnocchi', 'worth', 'gnocchi', 'it!', 'guy', 'point', 'Overall,', 'Overall', 'fast', 'Alfredo', '

In [0]:
stopwords = ['fantastic','go','beginning','reasonable','outstanding','good','expect','eat','beginning',]
def filter_stopwords(text):
  for i in str(text):
    if i not in stopwords:
      return str(text)

In [0]:
len(similar_list)

17840

In [0]:
for i in phrase_count_pivot['word_segments_unpacked']:
  model.get_nearest_neighbors(i)

[(0.9999875426292419, 'great.'),
 (0.9999858140945435, 'great'),
 (0.9999794363975525, 'since'),
 (0.9999735355377197, 'take'),
 (0.9999716877937317, 'our'),
 (0.9999716877937317, 'host'),
 (0.9999700784683228, 'like'),
 (0.9999678134918213, 'never'),
 (0.9999666213989258, 'it.'),
 (0.9999657273292542, 'long')]

###Word Analogies

In a similar spirit, one can play around with word analogies. For example, we can see if our model can guess what is to France, what Berlin is to Germany.

This can be done with the analogies functionality. It takes a word triplet (like Germany Berlin France) and outputs the analogy:

In [0]:
model.get_analogies('food', 'burnt','drink')

[(0.9998505711555481, 'Malay'),
 (0.9998475313186646, 'three'),
 (0.9998438358306885, 'about'),
 (0.9998437166213989, 'Malaysian'),
 (0.9998391270637512, 'think'),
 (0.9998390078544617, 'owners'),
 (0.9998306632041931, 'tasted'),
 (0.999829113483429, '"We'),
 (0.9998281598091125, 'will'),
 (0.9998226165771484, 'more')]

In [0]:
model.get_nearest_neighbors('gearshift')

[(0.9999086260795593, 'much'),
 (0.9999054670333862, 'favorite'),
 (0.99989914894104, 'around'),
 (0.9998939037322998, 'restaurants'),
 (0.9998882412910461, 'usually'),
 (0.999884307384491, 'delicious.'),
 (0.9998836517333984, 'just'),
 (0.9998722672462463, 'most'),
 (0.9998640418052673, 'here.'),
 (0.9998615980148315, "It's")]

In [0]:
model.get_word_vector("great")

array([ 0.4658365 ,  0.14228567, -0.0147468 , -0.07972424, -0.00482475,
        0.17291799, -0.1019064 ,  0.26551396,  0.38831973, -0.01296046,
       -0.24330784,  0.35275698, -0.18760549,  0.1195251 ,  0.0108976 ,
       -0.02055119,  0.18210387, -0.24476875,  0.32765675,  0.00889295,
       -0.2347713 ,  0.16657634,  0.13823693, -0.24581313,  0.20579338,
        0.36120856, -0.12453888, -0.04237   , -0.18034935,  0.28278336,
        0.1011708 ,  0.07667088, -0.21272978,  0.08931043, -0.0622963 ,
       -0.04397372,  0.2651232 , -0.22436944, -0.5980655 ,  0.06624931,
       -0.43059924,  0.18476237,  0.4062801 ,  0.476247  ,  0.21408293,
       -0.00188831,  0.17435957,  0.01167148, -0.12818441, -0.05374854,
       -0.17222664,  0.05946076, -0.2717286 ,  0.18880273, -0.0033478 ,
       -0.300269  , -0.0093135 ,  0.07271611,  0.00664317,  0.5401071 ,
       -0.11126462,  0.31273788,  0.16003864, -0.10227244,  0.0175306 ,
       -0.3204592 , -0.37458053,  0.15087034,  0.22499208, -0.20

In [0]:
model.get_nearest_neighbors('food')

NameError: ignored